* whale_siamese_submission.csv | ResNet50 with 150 epochs | 0.681 LB score | 0.93 cv score | batchsize 32
* whale_siamese_submission_v2.csv | DenseNet121 with 60 epochs | 0.329 | 0.96 cv score | batch size 16
* whale_siamese_submission_v3.csv | ResNet50 with 210 epochs | 0.794 | 0.991| batch size 32
* whale_siamese_submission_v4.csv | Xception with 210 epochs | 0.791 | 0.9997 | batch size 32
versions 1-4 are grayscale (128,128,1)

version 5

* whale_siamese_submission_v5.csv | basemodel with 210 epochs |  |  | batch size 32 | imagesize from 128 to 156




* whale_siamese_submission_v4.csv | ResNext with 250 epochs |  |  cv score | batch size 16
* whale_siamese_submission_v5.csv | ResNet50 with 300 epochs | | cv score | batch size 32
* whale_siamese_submission_v6.csv | Se-ResNext with 250 epochs | | cv score | batch size 16(8)
4: senet
5:resnext
6: se-resnext
7: Xception
8:InceptionResNetV2


Models to experiment: DenseNet, Xception, InceptionV3, InceptionResNetV2, senet, resnext

## TL;DR [0.681 with ResNet50 150 epochs]

Credits to Khoi Nguyen.
Things to try fixing "A smelly part is the DIY image augmentation, you can try albumetation or Keras's own apply_transform() utility."

I tried to refactor [@martinpiotte](https://kaggle.com/martinpiotte)'s original kernel [here](https://www.kaggle.com/martinpiotte/whale-recognition-model-with-score-0-78563).

I changed almost nothing beside commenting out the latter 380 epochs since it can't fit into a kernel. I also generated the new bounding boxes in my kernel [here](https://www.kaggle.com/suicaokhoailang/generating-whale-bounding-boxes) and saved it as a **.csv** instead of **pickle** for readability. 

A few things to point out:

- Training more will probably improve your score, maybe as many as 500 epochs.

- You may try to improve your training time by applying this technique (thanks **Brian**): https://www.kaggle.com/c/humpback-whale-identification/discussion/74402#444476 .

- Consider using a pretrained model(s), good for blending.



TO CONSIDER:
Try more sophisticated models such as resnext, possibly with helps from https://github.com/osmr/imgclsmob

In [1]:
import gc
gc.enable()
gc.collect()

35

In [2]:
#!pip install lapjv
import os
import math
# Read the dataset description
import gzip
# Read or generate p2h, a dictionary of image name to image id (picture to hash)
import pickle
import platform
import random
# Suppress annoying stderr output when importing keras.
import sys
from lapjv import lapjv
#from lap import lapjv
from math import sqrt
# Determine the size of each image
from os.path import isfile
import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image as pil_image
from imagehash import phash
from keras import backend as K
from keras import regularizers
from keras.engine.topology import Input
from keras.layers import Activation, Add, BatchNormalization, Concatenate, Conv2D, Dense, Flatten, GlobalMaxPooling2D, \
    Lambda, MaxPooling2D, Reshape
from keras import layers as nn
from keras.models import Model
from keras.optimizers import *
from keras.preprocessing.image import img_to_array
from keras.utils import Sequence
from pandas import read_csv
from scipy.ndimage import affine_transform
from tqdm import tqdm_notebook as tqdm

C:\Users\JoonH\AppData\Local\conda\conda\envs\TF\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
TRAIN_DF = '/Users/JoonH/Desktop/whale/train.csv'
SUB_Df = '/Users/JoonH/Desktop/whale/sample_submission.csv'
TRAIN = '/Users/JoonH/Desktop/whale/train_original/'
TEST = '/Users/JoonH/Desktop/whale/test/'
P2H = '/Users/JoonH/Desktop/whale/metadata/p2h.pickle'
P2SIZE = '/Users/JoonH/Desktop/whale/metadata/p2size.pickle'
BB_DF = "/Users/JoonH/Desktop/whale/metadata/bounding_boxes.csv"
tagged = dict([(p, w) for _, p, w in read_csv(TRAIN_DF).to_records()])
submit = [p for _, p, _ in read_csv(SUB_Df).to_records()]
join = list(tagged.keys()) + submit

In [4]:
def expand_path(p):
    if isfile(TRAIN + p):
        return TRAIN + p
    if isfile(TEST + p):
        return TEST + p
    return p

## Duplicate image identification

This part was from the original kernel, seems like in the playground competition dulicated images was a real issue. I don't know the case about this one but I took one for the team and generated the results anyway. I'm such a nice chap.

In [5]:
if isfile(P2SIZE):
    print("P2SIZE exists.")
    with open(P2SIZE, 'rb') as f:
        p2size = pickle.load(f)
else:
    p2size = {}
    for p in tqdm(join):
        size = pil_image.open(expand_path(p)).size
        p2size[p] = size

P2SIZE exists.


In [6]:
def match(h1, h2):
    for p1 in h2ps[h1]:
        for p2 in h2ps[h2]:
            i1 = pil_image.open(expand_path(p1))
            i2 = pil_image.open(expand_path(p2))
            if i1.mode != i2.mode or i1.size != i2.size: return False
            a1 = np.array(i1)
            a1 = a1 - a1.mean()
            a1 = a1 / sqrt((a1 ** 2).mean())
            a2 = np.array(i2)
            a2 = a2 - a2.mean()
            a2 = a2 / sqrt((a2 ** 2).mean())
            a = ((a1 - a2) ** 2).mean()
            if a > 0.1: return False
    return True


if isfile(P2H):
    print("P2H exists.")
    with open(P2H, 'rb') as f:
        p2h = pickle.load(f)
else:
    # Compute phash for each image in the training and test set.
    p2h = {}
    for p in tqdm(join):
        img = pil_image.open(expand_path(p))
        h = phash(img)
        p2h[p] = h

    # Find all images associated with a given phash value.
    h2ps = {}
    for p, h in p2h.items():
        if h not in h2ps: h2ps[h] = []
        if p not in h2ps[h]: h2ps[h].append(p)

    # Find all distinct phash values
    hs = list(h2ps.keys())

    # If the images are close enough, associate the two phash values (this is the slow part: n^2 algorithm)
    h2h = {}
    for i, h1 in enumerate(tqdm(hs)):
        for h2 in hs[:i]:
            if h1 - h2 <= 6 and match(h1, h2):
                s1 = str(h1)
                s2 = str(h2)
                if s1 < s2: s1, s2 = s2, s1
                h2h[s1] = s2

    # Group together images with equivalent phash, and replace by string format of phash (faster and more readable)
    for p, h in p2h.items():
        h = str(h)
        if h in h2h: h = h2h[h]
        p2h[p] = h
#     with open(P2H, 'wb') as f:
#         pickle.dump(p2h, f)
# For each image id, determine the list of pictures
h2ps = {}
for p, h in p2h.items():
    if h not in h2ps: h2ps[h] = []
    if p not in h2ps[h]: h2ps[h].append(p)

P2H exists.


In [7]:
def show_whale(imgs, per_row=2):
    n = len(imgs)
    rows = (n + per_row - 1) // per_row
    cols = min(per_row, n)
    fig, axes = plt.subplots(rows, cols, figsize=(24 // per_row * cols, 24 // per_row * rows))
    for ax in axes.flatten(): ax.axis('off')
    for i, (img, ax) in enumerate(zip(imgs, axes.flatten())): ax.imshow(img.convert('RGB'))
        

def read_raw_image(p):
    img = pil_image.open(expand_path(p))
    return img

In [8]:
# For each images id, select the prefered image
def prefer(ps):
    if len(ps) == 1: return ps[0]
    best_p = ps[0]
    best_s = p2size[best_p]
    for i in range(1, len(ps)):
        p = ps[i]
        s = p2size[p]
        if s[0] * s[1] > best_s[0] * best_s[1]:  # Select the image with highest resolution
            best_p = p
            best_s = s
    return best_p

h2p = {}
for h, ps in h2ps.items():
    h2p[h] = prefer(ps)
len(h2p), list(h2p.items())[:5]

(33317,
 [('d26698c3271c757c', '0000e88ab.jpg'),
  ('ba8cc231ad489b77', '0001f9222.jpg'),
  ('bbcad234a52d0f0b', '00029d126.jpg'),
  ('c09ae7dc09f33a29', '00050a15a.jpg'),
  ('d02f65ba9f74a08a', '0005c1ef8.jpg')])

In [9]:
# Read the bounding box data from the bounding box kernel (see reference above)
p2bb = pd.read_csv(BB_DF).set_index("Image")

old_stderr = sys.stderr
sys.stderr = open('/dev/null' if platform.system() != 'Windows' else 'nul', 'w')

sys.stderr = old_stderr

img_shape = (132, 132, 1)  # The image shape used by the model
#img_shape = (200, 200, 1) #grayscale version
img_shape2 = (128, 128, 1) # used during affine
anisotropy = 2.15  # The horizontal compression ratio
crop_margin = 0.05  # The margin added around the bounding box to compensate for bounding box inaccuracy

In [10]:
def build_transform(rotation, shear, height_zoom, width_zoom, height_shift, width_shift):
    """
    Build a transformation matrix with the specified characteristics.
    """
    rotation = np.deg2rad(rotation)
    shear = np.deg2rad(shear)
    rotation_matrix = np.array(
        [[np.cos(rotation), np.sin(rotation), 0], [-np.sin(rotation), np.cos(rotation), 0], [0, 0, 1]])
    shift_matrix = np.array([[1, 0, height_shift], [0, 1, width_shift], [0, 0, 1]])
    shear_matrix = np.array([[1, np.sin(shear), 0], [0, np.cos(shear), 0], [0, 0, 1]])
    zoom_matrix = np.array([[1.0 / height_zoom, 0, 0], [0, 1.0 / width_zoom, 0], [0, 0, 1]])
    shift_matrix = np.array([[1, 0, -height_shift], [0, 1, -width_shift], [0, 0, 1]])
    return np.dot(np.dot(rotation_matrix, shear_matrix), np.dot(zoom_matrix, shift_matrix))

In [11]:
def read_cropped_image(p, augment):
    """
    @param p : the name of the picture to read
    @param augment: True/False if data augmentation should be performed
    @return a numpy array with the transformed image
    """
    # If an image id was given, convert to filename
    if p in h2p:
        p = h2p[p]
    size_x, size_y = p2size[p]

    # Determine the region of the original image we want to capture based on the bounding box.
    row = p2bb.loc[p]
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    dx = x1 - x0
    dy = y1 - y0
    x0 -= dx * crop_margin
    x1 += dx * crop_margin + 1
    y0 -= dy * crop_margin
    y1 += dy * crop_margin + 1
    if x0 < 0:
        x0 = 0
    if x1 > size_x:
        x1 = size_x
    if y0 < 0:
        y0 = 0
    if y1 > size_y:
        y1 = size_y
    dx = x1 - x0
    dy = y1 - y0
    if dx > dy * anisotropy:
        dy = 0.5 * (dx / anisotropy - dy)
        y0 -= dy
        y1 += dy
    else:
        dx = 0.5 * (dy * anisotropy - dx)
        x0 -= dx
        x1 += dx

    # Generate the transformation matrix
    trans = np.array([[1, 0, -0.5 * img_shape[0]], [0, 1, -0.5 * img_shape[1]], [0, 0, 1]])
    trans = np.dot(np.array([[(y1 - y0) / img_shape[0], 0, 0], [0, (x1 - x0) / img_shape[1], 0], [0, 0, 1]]), trans)
    if augment:
        trans = np.dot(build_transform(
            random.uniform(-10, 10), #rotation
            random.uniform(-10, 10), #shear
            random.uniform(0.8, 1.0), #height_zoom
            random.uniform(0.8, 1.0), #width_zoom
            random.uniform(-0.05 * (y1 - y0), 0.05 * (y1 - y0)), #height_shift
            random.uniform(-0.05 * (x1 - x0), 0.05 * (x1 - x0)) #width_shift
        ), trans)
    trans = np.dot(np.array([[1, 0, 0.5 * (y1 + y0)], [0, 1, 0.5 * (x1 + x0)], [0, 0, 1]]), trans)

    # Read the image, transform to black and white and comvert to numpy array
    img = read_raw_image(p).convert('L')
    img = img_to_array(img)

    # Apply affine transformation
    matrix = trans[:2, :2]
    offset = trans[:2, 2]
    img = img.reshape(img.shape[:-1])
    img = affine_transform(img, matrix, offset, output_shape=img_shape[:-1], order=1, mode='constant',
                           cval=np.average(img))
    img = img.reshape(img_shape)

    # Normalize to zero mean and unit variance
    img -= np.mean(img, keepdims=True)
    img /= np.std(img, keepdims=True) + K.epsilon()
    return img

def read_for_training(p):
    """
    Read and preprocess an image with data augmentation (random transform).
    """
    return read_cropped_image(p, True)


def read_for_validation(p):
    """
    Read and preprocess an image without data augmentation (use for testing).
    """
    return read_cropped_image(p, False)


p = list(tagged.keys())[312]

In [12]:
def read_cropped_image_rgb(p, augment):
    """
    @param p : the name of the picture to read
    @param augment: True/False if data augmentation should be performed
    @return a numpy array with the transformed image
    """
    # If an image id was given, convert to filename
    if p in h2p:
        p = h2p[p]
    size_x, size_y = p2size[p]

    # Determine the region of the original image we want to capture based on the bounding box.
    row = p2bb.loc[p]
    x0, y0, x1, y1 = row['x0'], row['y0'], row['x1'], row['y1']
    dx = x1 - x0
    dy = y1 - y0
    x0 -= dx * crop_margin
    x1 += dx * crop_margin + 1
    y0 -= dy * crop_margin
    y1 += dy * crop_margin + 1
    if x0 < 0:
        x0 = 0
    if x1 > size_x:
        x1 = size_x
    if y0 < 0:
        y0 = 0
    if y1 > size_y:
        y1 = size_y
    dx = x1 - x0
    dy = y1 - y0
    if dx > dy * anisotropy:
        dy = 0.5 * (dx / anisotropy - dy)
        y0 -= dy
        y1 += dy
    else:
        dx = 0.5 * (dy * anisotropy - dx)
        x0 -= dx
        x1 += dx

    # Generate the transformation matrix
    trans = np.array([[1, 0, -0.5 * img_shape[0]], [0, 1, -0.5 * img_shape[1]], [0, 0, 1]])
    trans = np.dot(np.array([[(y1 - y0) / img_shape[0], 0, 0], [0, (x1 - x0) / img_shape[1], 0], [0, 0, 1]]), trans)
    if augment:
        trans = np.dot(build_transform(
            random.uniform(-5, 5),
            random.uniform(-5, 5),
            random.uniform(0.8, 1.0),
            random.uniform(0.8, 1.0),
            random.uniform(-0.05 * (y1 - y0), 0.05 * (y1 - y0)),
            random.uniform(-0.05 * (x1 - x0), 0.05 * (x1 - x0))
        ), trans)
    trans = np.dot(np.array([[1, 0, 0.5 * (y1 + y0)], [0, 1, 0.5 * (x1 + x0)], [0, 0, 1]]), trans)

    # Read the image, transform to black and white and comvert to numpy array
    img = read_raw_image(p).convert('RGB')
    img = img_to_array(img)
    #print(img.shape)
    # Apply affine transformation
    matrix = trans[:2, :2]
    offset = trans[:2, 2]
    #since we have RGB channels and scipy.ndimage.affine does not acknowledge spatial info,
    #we split the channels to perform affine and stack them back together.
    #print(img[:,:,0].shape)
    shape = img[:,:,0].shape + (1,)
    r = affine(img[:,:,0].reshape(shape), matrix, offset)
    g = affine(img[:,:,1].reshape(shape), matrix, offset)
    b = affine(img[:,:,2].reshape(shape), matrix, offset)
    return np.dstack((r,g,b))
    #img = img.reshape(img.shape[:-1])
    #img = affine_transform(img, matrix, offset, output_shape=img_shape[:-1], order=1, mode='constant',
    #                       cval=np.average(img))
    #img = img.reshape(img_shape)

    # Normalize to zero mean and unit variance
    #img -= np.mean(img, keepdims=True)
    #img /= np.std(img, keepdims=True) + K.epsilon()
    #return img
    

def affine(img, m, off):
    img = img.reshape(img.shape[:-1])
    #img = img.reshape(img.shape)
    img = affine_transform(img, m, off, output_shape=img_shape[:-1], order=1, mode='constant',
                           cval=np.average(img))
    img = img.reshape(img_shape2)

    # Normalize to zero mean and unit variance
    img -= np.mean(img, keepdims=True)
    img /= np.std(img, keepdims=True) + K.epsilon()
    return img

def read_for_training(p):
    """
    Read and preprocess an image with data augmentation (random transform).
    """
    return read_cropped_image(p, True)

def read_for_validation(p):
    """
    Read and preprocess an image without data augmentation (use for testing).
    """
    return read_cropped_image(p, False)


p = list(tagged.keys())[312]

In [17]:
from keras.regularizers import l2
def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            bn-activation-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    #x = AveragePooling2D(pool_size=8)(x)
    y = GlobalMaxPooling2D()(x)
    #y = Flatten()(x)
    #outputs = Dense(num_classes,
    #                activation='softmax',
    #                kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=y)
    return model

In [12]:
from keras import layers
def Xception(include_top=True,
             input_shape=None,
             pooling=None,
             classes=1000,
             reg = 0.0,
             **kwargs):
    """Instantiates the Xception architecture.
    Optionally loads weights pre-trained on ImageNet. This model can
    only be used with the data format `(width, height, channels)`.
    You should set `image_data_format='channels_last'` in your Keras config
    located at ~/.keras/keras.json.
    Note that the default input image size for this model is 299x299.
    # Arguments
        include_top: whether to include the fully-connected
            layer at the top of the network.
        weights: one of `None` (random initialization),
              'imagenet' (pre-training on ImageNet),
              or the path to the weights file to be loaded.
        input_tensor: optional Keras tensor
            (i.e. output of `layers.Input()`)
            to use as image input for the model.
        input_shape: optional shape tuple, only to be specified
            if `include_top` is False (otherwise the input shape
            has to be `(299, 299, 3)`.
            It should have exactly 3 inputs channels,
            and width and height should be no smaller than 71.
            E.g. `(150, 150, 3)` would be one valid value.
        pooling: Optional pooling mode for feature extraction
            when `include_top` is `False`.
            - `None` means that the output of the model will be
                the 4D tensor output of the
                last convolutional block.
            - `avg` means that global average pooling
                will be applied to the output of the
                last convolutional block, and thus
                the output of the model will be a 2D tensor.
            - `max` means that global max pooling will
                be applied.
        classes: optional number of classes to classify images
            into, only to be specified if `include_top` is True,
            and if no `weights` argument is specified.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
        RuntimeError: If attempting to run this model with a
            backend that does not support separable convolutions.
    """

    # Determine proper input shape

    img_input = layers.Input(shape=input_shape)

    x = layers.Conv2D(32, (3, 3),
                      strides=(2, 2),
                      use_bias=False,
                      name='block1_conv1')(img_input)
    x = layers.BatchNormalization(name='block1_conv1_bn')(x)
    x = layers.Activation('relu', name='block1_conv1_act')(x)
    x = layers.Conv2D(64, (3, 3), use_bias=False, name='block1_conv2',kernel_regularizer=l2(reg))(x)
    x = layers.BatchNormalization(name='block1_conv2_bn')(x)
    x = layers.Activation('relu', name='block1_conv2_act')(x)

    residual = layers.Conv2D(128, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False,
                             kernel_regularizer=l2(reg))(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv1')(x)
    x = layers.BatchNormalization(name='block2_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block2_sepconv2_act')(x)
    x = layers.SeparableConv2D(128, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block2_sepconv2')(x)
    x = layers.BatchNormalization(name='block2_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block2_pool')(x)
    x = layers.add([x, residual])

    residual = layers.Conv2D(256, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False, kernel_regularizer=l2(reg))(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.Activation('relu', name='block3_sepconv1_act')(x)
    x = layers.SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv1')(x)
    x = layers.BatchNormalization(name='block3_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block3_sepconv2_act')(x)
    x = layers.SeparableConv2D(256, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block3_sepconv2')(x)
    x = layers.BatchNormalization(name='block3_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block3_pool')(x)
    x = layers.add([x, residual])

    residual = layers.Conv2D(728, (1, 1),
                             strides=(2, 2),
                             padding='same',
                             use_bias=False,
                             kernel_regularizer=l2(reg))(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.Activation('relu', name='block4_sepconv1_act')(x)
    x = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv1')(x)
    x = layers.BatchNormalization(name='block4_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block4_sepconv2_act')(x)
    x = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block4_sepconv2')(x)
    x = layers.BatchNormalization(name='block4_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3), strides=(2, 2),
                            padding='same',
                            name='block4_pool')(x)
    x = layers.add([x, residual])

    for i in range(8):
        residual = x
        prefix = 'block' + str(i + 5)

        x = layers.Activation('relu', name=prefix + '_sepconv1_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv1')(x)
        x = layers.BatchNormalization(name=prefix + '_sepconv1_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv2_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv2')(x)
        x = layers.BatchNormalization(name=prefix + '_sepconv2_bn')(x)
        x = layers.Activation('relu', name=prefix + '_sepconv3_act')(x)
        x = layers.SeparableConv2D(728, (3, 3),
                                   padding='same',
                                   use_bias=False,
                                   name=prefix + '_sepconv3')(x)
        x = layers.BatchNormalization(name=prefix + '_sepconv3_bn')(x)

        x = layers.add([x, residual])

    residual = layers.Conv2D(1024, (1, 1), strides=(2, 2),
                             padding='same', use_bias=False,
                             kernel_regularizer=l2(reg))(x)
    residual = layers.BatchNormalization()(residual)

    x = layers.Activation('relu', name='block13_sepconv1_act')(x)
    x = layers.SeparableConv2D(728, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv1')(x)
    x = layers.BatchNormalization(name='block13_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block13_sepconv2_act')(x)
    x = layers.SeparableConv2D(1024, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block13_sepconv2')(x)
    x = layers.BatchNormalization(name='block13_sepconv2_bn')(x)

    x = layers.MaxPooling2D((3, 3),
                            strides=(2, 2),
                            padding='same',
                            name='block13_pool')(x)
    x = layers.add([x, residual])

    x = layers.SeparableConv2D(1536, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv1')(x)
    x = layers.BatchNormalization(name='block14_sepconv1_bn')(x)
    x = layers.Activation('relu', name='block14_sepconv1_act')(x)

    x = layers.SeparableConv2D(2048, (3, 3),
                               padding='same',
                               use_bias=False,
                               name='block14_sepconv2')(x)
    x = layers.BatchNormalization(name='block14_sepconv2_bn')(x)
    x = layers.Activation('relu', name='block14_sepconv2_act')(x)

    if include_top:
        x = layers.GlobalAveragePooling2D(name='avg_pool')(x)
        x = layers.Dense(classes, activation='softmax', name='predictions')(x)
    else:
        if pooling == 'avg':
            x = layers.GlobalAveragePooling2D()(x)
        elif pooling == 'max':
            x = layers.GlobalMaxPooling2D()(x)

    # Ensure that the model takes into account
    # any potential predecessors of `input_tensor`.
    inputs = img_input
    # Create model.
    model = Model(inputs, x, name='xception')

    # Load weights
    return model

In [39]:
# https://gist.github.com/mjdietzx/0cb95922aac14d446a6530f87b3a04ce
"""
Clean and simple Keras implementation of network architectures described in:
    - (ResNet-50) [Deep Residual Learning for Image Recognition](https://arxiv.org/pdf/1512.03385.pdf).
    - (ResNeXt-50 32x4d) [Aggregated Residual Transformations for Deep Neural Networks](https://arxiv.org/pdf/1611.05431.pdf).
    
Python 3.
"""

from keras import layers

#
# image dimensions
#

img_height = 132
img_width = 132
img_channels = 3

#
# network params
#

cardinality = 32


def residual_network(x, reg = 0.0):
    """
    ResNeXt by default. For ResNet set `cardinality` = 1 above.
    
    """
    def add_common_layers(y):
        y = layers.BatchNormalization()(y)
        y = layers.LeakyReLU()(y)

        return y

    def grouped_convolution(y, nb_channels, _strides, reg = reg):
        # when `cardinality` == 1 this is just a standard convolution
        if cardinality == 1:
            return layers.Conv2D(nb_channels, kernel_size=(3, 3), strides=_strides, padding='same')(y)
        
        assert not nb_channels % cardinality
        _d = nb_channels // cardinality

        # in a grouped convolution layer, input and output channels are divided into `cardinality` groups,
        # and convolutions are separately performed within each group
        groups = []
        for j in range(cardinality):
            group = layers.Lambda(lambda z: z[:, :, :, j * _d:j * _d + _d])(y)
            groups.append(layers.Conv2D(_d, kernel_size=(3, 3), strides=_strides, padding='same', kernel_regularizer=l2(reg))(group))
            
        # the grouped convolutional layer concatenates them as the outputs of the layer
        y = layers.concatenate(groups)

        return y

    def residual_block(y, nb_channels_in, nb_channels_out, _strides=(1, 1), _project_shortcut=False, reg = reg):
        """
        Our network consists of a stack of residual blocks. These blocks have the same topology,
        and are subject to two simple rules:

        - If producing spatial maps of the same size, the blocks share the same hyper-parameters (width and filter sizes).
        - Each time the spatial map is down-sampled by a factor of 2, the width of the blocks is multiplied by a factor of 2.
        """
        shortcut = y

        # we modify the residual building block as a bottleneck design to make the network more economical
        y = layers.Conv2D(nb_channels_in, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg))(y)
        y = add_common_layers(y)

        # ResNeXt (identical to ResNet when `cardinality` == 1)
        y = grouped_convolution(y, nb_channels_in, _strides=_strides, reg = reg)
        y = add_common_layers(y)

        y = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=(1, 1), padding='same', kernel_regularizer=l2(reg))(y)
        # batch normalization is employed after aggregating the transformations and before adding to the shortcut
        y = layers.BatchNormalization()(y)

        # identity shortcuts used directly when the input and output are of the same dimensions
        if _project_shortcut or _strides != (1, 1):
            # when the dimensions increase projection shortcut is used to match dimensions (done by 1×1 convolutions)
            # when the shortcuts go across feature maps of two sizes, they are performed with a stride of 2
            shortcut = layers.Conv2D(nb_channels_out, kernel_size=(1, 1), strides=_strides, padding='same', kernel_regularizer=l2(reg))(shortcut)
            shortcut = layers.BatchNormalization()(shortcut)

        y = layers.add([shortcut, y])

        # relu is performed right after each batch normalization,
        # expect for the output of the block where relu is performed after the adding to the shortcut
        y = layers.LeakyReLU()(y)

        return y

    # conv1
    x = layers.Conv2D(64, kernel_size=(7, 7), strides=(2, 2), padding='same', kernel_regularizer=l2(reg))(x)
    x = add_common_layers(x)

    # conv2
    x = layers.MaxPool2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    for i in range(3):
        project_shortcut = True if i == 0 else False
        x = residual_block(x, 128, 256, _project_shortcut=project_shortcut, reg = reg)

    # conv3
    for i in range(4):
        # down-sampling is performed by conv3_1, conv4_1, and conv5_1 with a stride of 2
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 256, 512, _strides=strides, reg = reg)

    # conv4
    for i in range(6):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 512, 1024, _strides=strides, reg = reg)

    # conv5
    for i in range(3):
        strides = (2, 2) if i == 0 else (1, 1)
        x = residual_block(x, 1024, 2048, _strides=strides, reg = reg)

    x = layers.GlobalAveragePooling2D()(x)
    # x = layers.Dense(1)(x)

    return x


#image_tensor = layers.Input(shape=(img_height, img_width, img_channels))
#network_output = residual_network(image_tensor)
  
#model = models.Model(inputs=[image_tensor], outputs=[network_output])
#print(model.summary())

In [43]:
from keras.regularizers import *
def subblock(x, filter, **kwargs):
    x = BatchNormalization()(x)
    y = x
    y = Conv2D(filter, (1, 1), activation='relu', **kwargs)(y)  # Reduce the number of features to 'filter'
    y = BatchNormalization()(y)
    y = Conv2D(filter, (3, 3), activation='relu', **kwargs)(y)  # Extend the feature field
    y = BatchNormalization()(y)
    y = Conv2D(K.int_shape(x)[-1], (1, 1), **kwargs)(y)  # no activation # Restore the number of original features
    y = Add()([x, y])  # Add the bypass connection
    y = Activation('relu')(y)
    return y


def build_model(lr, l2, activation='sigmoid'):
    K.clear_session()
    ##############
    # BRANCH MODEL
    ##############
    regul = regularizers.l2(l2)
    optim = Adamax(lr=lr)#Adam(lr=lr)
    kwargs = {'padding': 'same', 'kernel_regularizer': regul}
    
    '''inp = Input(shape=img_shape)  # 384x384x1
    x = Conv2D(64, (9, 9), strides=2, activation='elu', **kwargs)(inp)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 96x96x64
    for _ in range(2):
        x = BatchNormalization()(x)
        x = Conv2D(64, (3, 3), activation='elu', **kwargs)(x)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 48x48x64
    x = BatchNormalization()(x)
    x = Conv2D(128, (1, 1), activation='elu', **kwargs)(x)  # 48x48x128
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 24x24x128
    x = BatchNormalization()(x)
    x = Conv2D(256, (1, 1), activation='elu', **kwargs)(x)  # 24x24x256
    for _ in range(4):
        x = subblock(x, 64, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 12x12x256
    x = BatchNormalization()(x)
    x = Conv2D(384, (1, 1), activation='elu', **kwargs)(x)  # 12x12x384
    for _ in range(4):
        x = subblock(x, 96, **kwargs)

    x = MaxPooling2D((2, 2), strides=(2, 2))(x)  # 6x6x384
    x = BatchNormalization()(x)
    x = Conv2D(512, (1, 1), activation='elu', **kwargs)(x)  # 6x6x512
    for _ in range(4):
        x = subblock(x, 128, **kwargs)

    x = GlobalMaxPooling2D()(x)  # 512
    branch_model = Model(inp, x)'''
    
    #from keras.applications.xception import Xception
    #from keras.applications.resnet50 import ResNet50
    #branch_model = ResNet50(input_shape = img_shape, include_top = False, weights = None, pooling='max')
    #branch_model = Xception(include_top=False,input_shape=img_shape, pooling='max', reg = l2)
    image_tensor = Input(shape=(img_height, img_width, 1))
    network_output = residual_network(image_tensor, reg = l2)
    branch_model = Model(inputs=[image_tensor], outputs=[network_output])
    #for layer in branch_model.layers:
    #    layer.trainable = True
    #branch_model = Xception(input_shape = img_shape, include_top = False, weights = None, pooling='max')
    #branch_model =  ResNext(input_shape=img_shape, depth=20, cardinality=5, width=64, weight_decay=5e-4,
    #        include_top=False, weights=None, input_tensor=None,
    #        pooling='max')
    ############
    # HEAD MODEL
    ############
    mid = 32
    xa_inp = Input(shape=branch_model.output_shape[1:])
    xb_inp = Input(shape=branch_model.output_shape[1:])
    x1 = Lambda(lambda x: x[0] * x[1])([xa_inp, xb_inp])
    x2 = Lambda(lambda x: x[0] + x[1])([xa_inp, xb_inp])
    x3 = Lambda(lambda x: K.abs(x[0] - x[1]))([xa_inp, xb_inp])
    x4 = Lambda(lambda x: K.square(x))(x3)
    x = Concatenate()([x1, x2, x3, x4])
    x = Reshape((4, branch_model.output_shape[1], 1), name='reshape1')(x)

    # Per feature NN with shared weight is implemented using CONV2D with appropriate stride.
    x = Conv2D(mid, (4, 1), activation='elu', padding='valid')(x)
    x = Reshape((branch_model.output_shape[1], mid, 1))(x)
    x = Conv2D(1, (1, mid), activation='linear', padding='valid')(x)
    x = Flatten(name='flatten')(x)

    # Weighted sum implemented as a Dense layer.
    x = Dense(1, use_bias=True, activation=activation, name='weighted-average')(x)
    head_model = Model([xa_inp, xb_inp], x, name='head')

    ########################
    # SIAMESE NEURAL NETWORK
    ########################
    # Complete model is constructed by calling the branch model on each input image,
    # and then the head model on the resulting 512-vectors.
    img_a = Input(shape=img_shape)
    img_b = Input(shape=img_shape)
    xa = branch_model(img_a)
    xb = branch_model(img_b)
    x = head_model([xa, xb])
    model = Model([img_a, img_b], x)
    model.compile(optim, loss='binary_crossentropy', metrics=['binary_crossentropy', 'acc'])
    
    for layer in model.layers:
        layer.trainable = True
    return model, branch_model, head_model


model, branch_model, head_model = build_model(64e-5, 0)

In [44]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 132, 132, 1)  0                                            
__________________________________________________________________________________________________
input_5 (InputLayer)            (None, 132, 132, 1)  0                                            
__________________________________________________________________________________________________
model_1 (Model)                 (None, 2048)         23075968    input_4[0][0]                    
                                                                 input_5[0][0]                    
__________________________________________________________________________________________________
head (Model)                    (None, 1)            2242        model_1[1][0]                    
          

In [15]:
branch_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 324, 324, 1)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 162, 162, 64) 5248        input_1[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 81, 81, 64)   0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 81, 81, 64)   256         max_pooling2d_1[0][0]            
__________________________________________________________________________________________________
conv2d_2 (

__________________________________________________________________________________________________
max_pooling2d_4 (MaxPooling2D)  (None, 10, 10, 256)  0           activation_8[0][0]               
__________________________________________________________________________________________________
batch_normalization_29 (BatchNo (None, 10, 10, 256)  1024        max_pooling2d_4[0][0]            
__________________________________________________________________________________________________
conv2d_30 (Conv2D)              (None, 10, 10, 384)  98688       batch_normalization_29[0][0]     
__________________________________________________________________________________________________
batch_normalization_30 (BatchNo (None, 10, 10, 384)  1536        conv2d_30[0][0]                  
__________________________________________________________________________________________________
conv2d_31 (Conv2D)              (None, 10, 10, 96)   36960       batch_normalization_30[0][0]     
__________

In [16]:
head_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 512)          0                                            
__________________________________________________________________________________________________
lambda_3 (Lambda)               (None, 512)          0           input_2[0][0]                    
                                                                 input_3[0][0]                    
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 512)          0           input_2[0][0]                    
          

In [15]:
h2ws = {}
new_whale = 'new_whale'
for p, w in tagged.items():
    if w != new_whale:  # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
for h, ws in h2ws.items():
    if len(ws) > 1:
        h2ws[h] = sorted(ws)

# For each whale, find the unambiguous images ids.
w2hs = {}
for h, ws in h2ws.items():
    if len(ws) == 1:  # Use only unambiguous pictures
        w = ws[0]
        if w not in w2hs: w2hs[w] = []
        if h not in w2hs[w]: w2hs[w].append(h)
for w, hs in w2hs.items():
    if len(hs) > 1:
        w2hs[w] = sorted(hs)

In [16]:
train = []  # A list of training image ids
for hs in w2hs.values():
    if len(hs) > 1:
        train += hs
random.shuffle(train)
train_set = set(train)

w2ts = {}  # Associate the image ids from train to each whale id.
for w, hs in w2hs.items():
    for h in hs:
        if h in train_set:
            if w not in w2ts:
                w2ts[w] = []
            if h not in w2ts[w]:
                w2ts[w].append(h)
for w, ts in w2ts.items():
    w2ts[w] = np.array(ts)

t2i = {}  # The position in train of each training image id
for i, t in enumerate(train):
    t2i[t] = i

In [45]:
import threading

class TrainingData(Sequence):
    def __init__(self, score, steps=1000, batch_size=8): #batchsize = 32
        """
        @param score the cost matrix for the picture matching
        @param steps the number of epoch we are planning with this score matrix
        """
        super(TrainingData, self).__init__()
        self.score = -score  # Maximizing the score is the same as minimuzing -score.
        self.steps = steps
        self.batch_size = batch_size
        for ts in w2ts.values():
            idxs = [t2i[t] for t in ts]
            for i in idxs:
                for j in idxs:
                    self.score[
                        i, j] = 10000.0  # Set a large value for matching whales -- eliminates this potential pairing
        self.on_epoch_end()

    def __getitem__(self, index):
        start = self.batch_size * index
        end = min(start + self.batch_size, len(self.match) + len(self.unmatch))
        size = end - start
        assert size > 0
        a = np.zeros((size,) + img_shape, dtype=K.floatx())
        b = np.zeros((size,) + img_shape, dtype=K.floatx())
        c = np.zeros((size, 1), dtype=K.floatx())
        j = start // 2
        for i in range(0, size, 2):
            a[i, :, :, :] = read_for_training(self.match[j][0])
            b[i, :, :, :] = read_for_training(self.match[j][1])
            c[i, 0] = 1  # This is a match
            a[i + 1, :, :, :] = read_for_training(self.unmatch[j][0])
            b[i + 1, :, :, :] = read_for_training(self.unmatch[j][1])
            c[i + 1, 0] = 0  # Different whales
            j += 1
        return [a, b], c

    def on_epoch_end(self):
        if self.steps <= 0: return  # Skip this on the last epoch.
        self.steps -= 1
        self.match = []
        self.unmatch = []
        #Note: Martin uses y to assign scores, but it is likely that x will hold same value as y
        # lap outputs cost,x,y while lapjv outputs x,y,cost
        #_, _,x  = lapjv(self.score) use this in case of lap
        #_, x, _  = lapjv(self.score) use this one for lapjv.lapjv
        #_, _, x  = lapjv(self.score) # Solve the linear assignment problem
        #y = np.arange(len(x), dtype=np.int32)
        
        #==============threading code starts here============
        #Also note, threading essentially splits the set into multiple for comparison, meaning more splits by getting more threads
        # will essentially lead to more inaccurate comparisons. do not use the threading solution for best results.
        num_threads = 3
        tmp   = num_threads*[None]
        threads   = []
        thread_input   = num_threads*[None]
        thread_idx = 0
        batch = score.shape[0] // (num_threads-1)
        for start in range(0, score.shape[0], batch):
            end = min(score.shape[0], start + batch)
            thread_input[thread_idx]  = self.score[start:end, start:end]
            thread_idx += 1

        def worker(data_idx):
            #x,_,_ = lapjv(thread_input[data_idx])
            _,x,_ = lapjv(thread_input[data_idx])
            tmp[data_idx] = x

        #print("Start worker threads")
        for i in range(num_threads):
            t = threading.Thread(target=worker, args=(i,), daemon=True)
            t.start()
            threads.append(t)
        for t in threads:
            if t is not None:
                t.join()
        x = np.concatenate(tmp)
        #print("LAP completed")
        y = np.arange(len(x), dtype=np.int32)
        #y = np.arange(x, dtype=np.int32)S
        #==============thrading section ends here=
        
        # Compute a derangement for matching whales
        for ts in w2ts.values():
            d = ts.copy()
            while True:
                random.shuffle(d)
                if not np.any(ts == d): break
            for ab in zip(ts, d): self.match.append(ab)

        # Construct unmatched whale pairs from the LAP solution.
        for i, j in zip(x, y):
            if i == j:
                print(self.score)
                print(x)
                print(y)
                print(i, j)
            #assert i != j
            self.unmatch.append((train[i], train[j]))

        # Force a different choice for an eventual next epoch.
        self.score[x, y] = 10000.0
        self.score[y, x] = 10000.0
        random.shuffle(self.match)
        random.shuffle(self.unmatch)
        # print(len(self.match), len(train), len(self.unmatch), len(train))
        assert len(self.match) == len(train) and len(self.unmatch) == len(train)

    def __len__(self):
        return (len(self.match) + len(self.unmatch) + self.batch_size - 1) // self.batch_size


# Test on a batch of 32 with random costs.
score = np.random.random_sample(size=(len(train), len(train)))
data = TrainingData(score)
(a, b), c = data[0]

In [18]:
# A Keras generator to evaluate only the BRANCH MODEL
class FeatureGen(Sequence):
    def __init__(self, data, batch_size=64, verbose=1):
        super(FeatureGen, self).__init__()
        self.data = data
        self.batch_size = batch_size
        self.verbose = verbose
        if self.verbose > 0: self.progress = tqdm(total=len(self), desc='Features')

    def __getitem__(self, index):
        start = self.batch_size * index
        size = min(len(self.data) - start, self.batch_size)
        a = np.zeros((size,) + img_shape, dtype=K.floatx())
        for i in range(size): a[i, :, :, :] = read_for_validation(self.data[start + i])
        if self.verbose > 0:
            self.progress.update()
            if self.progress.n >= len(self): self.progress.close()
        return a

    def __len__(self):
        return (len(self.data) + self.batch_size - 1) // self.batch_size


class ScoreGen(Sequence):
    def __init__(self, x, y=None, batch_size=2048, verbose=1):
        super(ScoreGen, self).__init__()
        self.x = x
        self.y = y
        self.batch_size = batch_size
        self.verbose = verbose
        if y is None:
            self.y = self.x
            self.ix, self.iy = np.triu_indices(x.shape[0], 1)
        else:
            self.iy, self.ix = np.indices((y.shape[0], x.shape[0]))
            self.ix = self.ix.reshape((self.ix.size,))
            self.iy = self.iy.reshape((self.iy.size,))
        self.subbatch = (len(self.x) + self.batch_size - 1) // self.batch_size
        if self.verbose > 0:
            self.progress = tqdm(total=len(self), desc='Scores')

    def __getitem__(self, index):
        start = index * self.batch_size
        end = min(start + self.batch_size, len(self.ix))
        a = self.y[self.iy[start:end], :]
        b = self.x[self.ix[start:end], :]
        if self.verbose > 0:
            self.progress.update()
            if self.progress.n >= len(self): self.progress.close()
        return [a, b]

    def __len__(self):
        return (len(self.ix) + self.batch_size - 1) // self.batch_size


In [21]:
def set_lr(model, lr):
    K.set_value(model.optimizer.lr, float(lr))

def get_lr(model):
    return K.get_value(model.optimizer.lr)


def score_reshape(score, x, y=None):
    """
    Tranformed the packed matrix 'score' into a square matrix.
    @param score the packed matrix
    @param x the first image feature tensor
    @param y the second image feature tensor if different from x
    @result the square matrix
    """
    if y is None:
        # When y is None, score is a packed upper triangular matrix.
        # Unpack, and transpose to form the symmetrical lower triangular matrix.
        m = np.zeros((x.shape[0], x.shape[0]), dtype=K.floatx())
        m[np.triu_indices(x.shape[0], 1)] = score.squeeze()
        m += m.transpose()
    else:
        m = np.zeros((y.shape[0], x.shape[0]), dtype=K.floatx())
        iy, ix = np.indices((y.shape[0], x.shape[0]))
        ix = ix.reshape((ix.size,))
        iy = iy.reshape((iy.size,))
        m[iy, ix] = score.squeeze()
    return m


def compute_score(verbose=1):
    """
    Compute the score matrix by scoring every pictures from the training set against every other picture O(n^2).
    """
    features = branch_model.predict_generator(FeatureGen(train, verbose=verbose), max_queue_size=12, workers=6,
                                              verbose=0)
    score = head_model.predict_generator(ScoreGen(features, verbose=verbose), max_queue_size=12, workers=6, verbose=0)
    score = score_reshape(score, features)
    return features, score

def make_steps(step, ampl):
    """
    Perform training epochs
    @param step Number of epochs to perform
    @param ampl the K, the randomized component of the score matrix.
    """
    global w2ts, t2i, steps, features, score, histories

    # shuffle the training pictures
    random.shuffle(train)

    # Map whale id to the list of associated training picture hash value
    w2ts = {}
    for w, hs in w2hs.items():
        for h in hs:
            if h in train_set:
                if w not in w2ts: w2ts[w] = []
                if h not in w2ts[w]: w2ts[w].append(h)
    for w, ts in w2ts.items(): w2ts[w] = np.array(ts)

    # Map training picture hash value to index in 'train' array    
    t2i = {}
    for i, t in enumerate(train): t2i[t] = i

    # Compute the match score for each picture pair
    features, score = compute_score()
    
    # Train the model for 'step' epochs
    history = model.fit_generator(
        TrainingData(score + ampl * np.random.random_sample(size=score.shape), steps=step, batch_size=16),
        initial_epoch=steps, epochs=steps + step, max_queue_size=12, workers=6, verbose=2).history
    steps += step

    # Collect history data
    history['epochs'] = steps
    history['ms'] = np.mean(score)
    history['lr'] = get_lr(model)
    print(history['epochs'], history['lr'], history['ms'])
    histories.append(history)

In [46]:
gc.collect()

71

In [48]:
#model.load_weights("/Users/JoonH/models/martin-piotte-v3epoch210.hdf5", by_name = True)

In [47]:
histories = []
steps = 0
version = 6

if (False): #isfile('./models/standard.model'):
    tmp = keras.models.load_model('./models/standard.model')
    model.set_weights(tmp.get_weights())
else:
    
# epoch -> 10
    make_steps(10,1000)
    ampl = 100.0
# epoch -> 60
    for _ in range (10):
        make_steps(5,ampl)
        ampl = max(1.0, 100 ** -0.1 * ampl)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch60.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'eppch60.hdf5')
    gc.collect()
    # epoch --> 110
    for _ in range(10): make_steps(5, 1.0)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch110.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch110.hdf5')
    gc.collect()
#     # epoch -> 150
    for _ in range(8): make_steps(5, 1.0)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch150.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch150.hdf5')
    gc.collect()
#     # epoch -> 200
    set_lr(model, 16e-5)
    for _ in range(10): make_steps(5, 0.5)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch200.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch200.hdf5')
    gc.collect()
    #model.load_weights("/Users/JoonH/models/martin-piotte-v3epoch210.hdf5")
     # epoch -> 240
    set_lr(model, 4e-5)
    for _ in range(8): make_steps(5, 0.25)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch240.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch240.hdf5')
    gc.collect()
     # epoch -> 250
    set_lr(model, 1e-5)
    for _ in range(2): make_steps(5, 0.25)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch250.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch250.hdf5')
    gc.collect()
     # epoch -> 300
    model, branch_model, head_model = build_model(64e-5, 0.0002)
    model.load_weights("/Users/JoonH/models/martin-piotte-v6epoch250.hdf5")
    for _ in range(10): make_steps(5, 1.0)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch300.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch300.hdf5')
    gc.collect()
     # epoch -> 350
    set_lr(model, 16e-5)
    for _ in range(10): make_steps(5, 0.5)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch350.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch350.hdf5')
    gc.collect()
     # epoch -> 390
    set_lr(model, 4e-5)
    for _ in range(8): make_steps(5, 0.25)
    model.save('/Users/JoonH/models/standard' + str(version) +'epoch390.model')
    model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch390.hdf5')
    gc.collect()
     # epoch -> 400
    set_lr(model, 1e-5)
    for _ in range(2): make_steps(5, 0.25)
    model.save('/Users/JoonH/models//standard.model')


Epoch 1/10
 - 1376s - loss: 0.5796 - binary_crossentropy: 0.5796 - acc: 0.6815
Epoch 2/10
 - 1326s - loss: 0.4902 - binary_crossentropy: 0.4902 - acc: 0.7604
Epoch 3/10
 - 1326s - loss: 0.4506 - binary_crossentropy: 0.4506 - acc: 0.7877
Epoch 4/10
 - 1326s - loss: 0.4221 - binary_crossentropy: 0.4221 - acc: 0.8068
Epoch 5/10
 - 1326s - loss: 0.3963 - binary_crossentropy: 0.3963 - acc: 0.8231
Epoch 6/10
 - 1326s - loss: 0.3731 - binary_crossentropy: 0.3731 - acc: 0.8305
Epoch 7/10
 - 1326s - loss: 0.3492 - binary_crossentropy: 0.3492 - acc: 0.8478
Epoch 8/10
 - 1327s - loss: 0.3329 - binary_crossentropy: 0.3329 - acc: 0.8547
Epoch 9/10
 - 1326s - loss: 0.3192 - binary_crossentropy: 0.3192 - acc: 0.8633
Epoch 10/10
 - 1327s - loss: 0.3142 - binary_crossentropy: 0.3142 - acc: 0.8653
10 0.00064 0.4990483



Epoch 11/15
 - 1326s - loss: 0.3011 - binary_crossentropy: 0.3011 - acc: 0.8726
Epoch 12/15
 - 1328s - loss: 0.2897 - binary_crossentropy: 0.2897 - acc: 0.8783
Epoch 13/15
 - 1327s - loss: 0.2791 - binary_crossentropy: 0.2791 - acc: 0.8850
Epoch 14/15
 - 1327s - loss: 0.2751 - binary_crossentropy: 0.2751 - acc: 0.8855
Epoch 15/15
 - 1327s - loss: 0.2710 - binary_crossentropy: 0.2710 - acc: 0.8840
15 0.00064 0.9833429



Epoch 16/20
 - 1326s - loss: 0.2622 - binary_crossentropy: 0.2622 - acc: 0.8901
Epoch 17/20
 - 1328s - loss: 0.2591 - binary_crossentropy: 0.2591 - acc: 0.8930
Epoch 18/20
 - 1327s - loss: 0.2480 - binary_crossentropy: 0.2480 - acc: 0.8968
Epoch 19/20
 - 1328s - loss: 0.2414 - binary_crossentropy: 0.2414 - acc: 0.9019
Epoch 20/20
 - 1328s - loss: 0.2339 - binary_crossentropy: 0.2339 - acc: 0.9031
20 0.00064 0.9881846



Epoch 21/25
 - 1331s - loss: 0.2391 - binary_crossentropy: 0.2391 - acc: 0.9008
Epoch 22/25
 - 1334s - loss: 0.2291 - binary_crossentropy: 0.2291 - acc: 0.9058
Epoch 23/25
 - 1334s - loss: 0.2226 - binary_crossentropy: 0.2226 - acc: 0.9075
Epoch 24/25
 - 1334s - loss: 0.2193 - binary_crossentropy: 0.2193 - acc: 0.9118
Epoch 25/25
 - 1334s - loss: 0.2144 - binary_crossentropy: 0.2144 - acc: 0.9120
25 0.00064 0.9978457



Epoch 26/30
 - 1333s - loss: 0.2191 - binary_crossentropy: 0.2191 - acc: 0.9110
Epoch 27/30
 - 1334s - loss: 0.2065 - binary_crossentropy: 0.2065 - acc: 0.9155
Epoch 28/30
 - 1334s - loss: 0.2048 - binary_crossentropy: 0.2048 - acc: 0.9165
Epoch 29/30
 - 1334s - loss: 0.2002 - binary_crossentropy: 0.2002 - acc: 0.9179
Epoch 30/30
 - 1334s - loss: 0.1955 - binary_crossentropy: 0.1955 - acc: 0.9206
30 0.00064 0.99811924


...


Epoch 376/380
 - 1372s - loss: 0.1833 - binary_crossentropy: 0.1303 - acc: 0.9506
Epoch 377/380
 - 1371s - loss: 0.1808 - binary_crossentropy: 0.1280 - acc: 0.9513
Epoch 378/380
 - 1372s - loss: 0.1755 - binary_crossentropy: 0.1227 - acc: 0.9527
Epoch 379/380
 - 1372s - loss: 0.1718 - binary_crossentropy: 0.1192 - acc: 0.9541
Epoch 380/380
 - 1373s - loss: 0.1709 - binary_crossentropy: 0.1184 - acc: 0.9545
380 4e-05 0.9999934



Epoch 381/385
 - 1373s - loss: 0.1754 - binary_crossentropy: 0.1230 - acc: 0.9528
Epoch 382/385
 - 1374s - loss: 0.1736 - binary_crossentropy: 0.1212 - acc: 0.9535
Epoch 383/385
 - 1374s - loss: 0.1723 - binary_crossentropy: 0.1200 - acc: 0.9537
Epoch 384/385
 - 1374s - loss: 0.1725 - binary_crossentropy: 0.1204 - acc: 0.9530
Epoch 385/385
 - 1374s - loss: 0.1712 - binary_crossentropy: 0.1191 - acc: 0.9548
385 4e-05 0.0



Epoch 386/390
 - 1373s - loss: 0.1742 - binary_crossentropy: 0.1221 - acc: 0.9535
Epoch 387/390
 - 1371s - loss: 0.1702 - binary_crossentropy: 0.1183 - acc: 0.9552
Epoch 388/390
 - 1371s - loss: 0.1709 - binary_crossentropy: 0.1190 - acc: 0.9556
Epoch 389/390
 - 1372s - loss: 0.1686 - binary_crossentropy: 0.1168 - acc: 0.9552
Epoch 390/390
 - 1372s - loss: 0.1667 - binary_crossentropy: 0.1149 - acc: 0.9563
390 4e-05 0.9999934



Epoch 391/395
 - 1366s - loss: 0.1658 - binary_crossentropy: 0.1141 - acc: 0.9566
Epoch 392/395
 - 1368s - loss: 0.1679 - binary_crossentropy: 0.1163 - acc: 0.9563
Epoch 393/395
 - 1368s - loss: 0.1638 - binary_crossentropy: 0.1123 - acc: 0.9576
Epoch 394/395
 - 1369s - loss: 0.1587 - binary_crossentropy: 0.1071 - acc: 0.9605
Epoch 395/395
 - 1368s - loss: 0.1578 - binary_crossentropy: 0.1063 - acc: 0.9591
395 1e-05 0.0



Epoch 396/400
 - 1368s - loss: 0.1591 - binary_crossentropy: 0.1077 - acc: 0.9602
Epoch 397/400
 - 1368s - loss: 0.1613 - binary_crossentropy: 0.1099 - acc: 0.9590
Epoch 398/400
 - 1368s - loss: 0.1589 - binary_crossentropy: 0.1076 - acc: 0.9592
Epoch 399/400
 - 1369s - loss: 0.1561 - binary_crossentropy: 0.1048 - acc: 0.9617
Epoch 400/400
 - 1368s - loss: 0.1594 - binary_crossentropy: 0.1081 - acc: 0.9597
400 1e-05 0.9999934


In [48]:
#optional, save weights after the training
#version = 3
#model.save('./models/standard' + str(version) +'400.model')
model.save_weights('/Users/JoonH/models/martin-piotte-v' + str(version) + 'epoch400.hdf5')

In [53]:
model.load_weights("/Users/JoonH/models/martin-piotte-v6epoch250.hdf5")

In [48]:
#tmp = keras.models.load_model('./models/standard4epoch200.model')
#model.set_weights(tmp.get_weights())
#model = keras.models.load_model('./models/standard4epoch160.model')

In [49]:
def prepare_submission(threshold, filename):
    """
    Generate a Kaggle submission file.
    @param threshold the score given to 'new_whale'
    @param filename the submission file name
    """
    vtop = 0
    vhigh = 0
    pos = [0, 0, 0, 0, 0, 0]
    with open(filename, 'wt', newline='\n') as f:
        f.write('Image,Id\n')
        for i, p in enumerate(tqdm(submit)):
            t = []
            s = set()
            a = score[i, :]
            for j in list(reversed(np.argsort(a))):
                h = known[j]
                if a[j] < threshold and new_whale not in s:
                    pos[len(t)] += 1
                    s.add(new_whale)
                    t.append(new_whale)
                    if len(t) == 5: break;
                for w in h2ws[h]:
                    assert w != new_whale
                    if w not in s:
                        if a[j] > 1.0:
                            vtop += 1
                        elif a[j] >= threshold:
                            vhigh += 1
                        s.add(w)
                        t.append(w)
                        if len(t) == 5: break;
                if len(t) == 5: break;
            if new_whale not in s: pos[5] += 1
            assert len(t) == 5 and len(s) == 5
            f.write(p + ',' + ' '.join(t[:5]) + '\n')
    return vtop, vhigh, pos

In [54]:
# Find elements from training sets not 'new_whale'
h2ws = {}
for p, w in tagged.items():
    if w != new_whale:  # Use only identified whales
        h = p2h[p]
        if h not in h2ws: h2ws[h] = []
        if w not in h2ws[h]: h2ws[h].append(w)
known = sorted(list(h2ws.keys()))

# Dictionary of picture indices
h2i = {}
for i, h in enumerate(known): h2i[h] = i

# Evaluate the model.
fknown = branch_model.predict_generator(FeatureGen(known), max_queue_size=20, workers=10, verbose=0)
fsubmit = branch_model.predict_generator(FeatureGen(submit), max_queue_size=20, workers=10, verbose=0)
score = head_model.predict_generator(ScoreGen(fknown, fsubmit), max_queue_size=20, workers=10, verbose=0)
score = score_reshape(score, fknown, fsubmit)

# Generate the subsmission file.
#prepare_submission(0.99, 'whale_siamese_submission_v' + str(version) + '.csv')
prepare_submission(0.99, 'whale_siamese_submission_v6_epoch260.csv')

(0, 15246, [3351, 1565, 315, 216, 162, 2351])

In [55]:
prepare_submission(0.98, 'whale_siamese_submission_v6_epoch260_threhold98.csv')

(0, 39800, [0, 0, 0, 0, 0, 7960])

In [56]:
prepare_submission(0.95, 'whale_siamese_submission_v6_epoch260_threshold95.csv')

(0, 39800, [0, 0, 0, 0, 0, 7960])